In [1]:
import cv2
import mediapipe as mp
import tensorflow as tf
import vgamepad as vg

In [3]:
# Might need to comment the check in base_optimizer.py (keras)
model = tf.keras.models.load_model('models/gesture_model.h5')

c:\Users\benja\Documents\video_game_gesture_control\env_vg_gesture_control\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [4]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

In [5]:
running = True
cap = cv2.VideoCapture(0)

In [6]:
while running:
	success, frame = cap.read()
	if not success:
		break
	frame = cv2.flip(frame, 1)
	results = hands.process(frame)
	if results.multi_hand_landmarks:
		for hand, hand_landmarks in zip(results.multi_handedness, results.multi_hand_landmarks):
			index_finger_landmarks = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
			index_finger_x = int(index_finger_landmarks.x * frame.shape[1])
			index_finger_y = int(index_finger_landmarks.y * frame.shape[0])
			if index_finger_x < frame.shape[1] // 6:
				gesture_x = 'Left'
			elif index_finger_x < frame.shape[1] // 2 and index_finger_x > frame.shape[1] * 1 // 3:
				gesture_x = 'Right'
			else:
				gesture_x = 'Neutral'
			if index_finger_y < frame.shape[0] // 3:
				gesture_y = 'Up'
			elif index_finger_y > frame.shape[0] * 2 // 3:
				gesture_y = 'Down'
			else:
				gesture_y = 'Neutral'
			if hand.classification[0].label == 'Left' and gesture_x == 'Left' and gesture_y == 'Up':
				print('left')
			if hand.classification[0].label == 'Right' and gesture_x == 'Left' and gesture_y == 'Up':
				print('right')
			cv2.circle(frame, (index_finger_x, index_finger_y), 10, (0, 255, 0), -1)
			cv2.putText(frame, gesture_x + gesture_y, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
	cv2.imshow('Gesture Recognition', frame)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

c:\Users\benja\Documents\video_game_gesture_control\env_vg_gesture_control\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


https://github.com/yannbouteiller/vgamepad